In [1]:
from aicsimageio import AICSImage
from skimage.io import imshow
from matplotlib import pyplot as plt
import numpy as np
from skimage import transform, draw
import xml.etree.ElementTree as ET
from matplotlib.colors import LinearSegmentedColormap

In [14]:
lst = open("/scratch/c.c23115040/H&E_Annotated/list.txt", "r")
lst = lst.read()
lst = lst.split("\n")

In [51]:
lst2 = open("/scratch/c.c23115040/H&E_Annotated2/list.txt", "r")
lst2 = lst2.read()
lst2 = lst2.split("\n")

In [2]:
lst3 = open("/scratch/c.c23115040/WCB_Sequential_Annotated/list.txt", "r")
lst3 = lst3.read()
lst3 = lst3.split("\n")

In [2]:
lst4 = open("/scratch/c.c23115040/MGP/D&E Annotated/list.txt", "r")
lst4 = lst4.read()
lst4 = lst4.split("\n")

In [3]:
def extract_annotations_from_metadata(metadata, scale):
    root = metadata[0]
    masks_healthy = []  
    masks_gleason3 = []
    masks_gleason4 = []
    masks_cribriform = []
    masks_glomeruloid = []
    masks_gleason5 = []
    #annotations = {} 
    img_width = round(float(root.find('./Information/Image/SizeX').text)*scale)
    img_height = round(float(root.find('./Information/Image/SizeY').text)*scale)

     #Contour_polygon
    for polygon in root.iter('Polygon'):
        try:
            color = polygon.find('./Attributes/Stroke').text
        except:
            continue
        points = polygon.find('./Geometry/Points').text
        array = np.array(np.mat(points.replace(' ', ';'))*scale, dtype=np.int64)
        r = array[:,1]
        c = array[:,0]
        img_polygon = np.zeros((img_height, img_width), dtype=np.int64)
        try:
            rotation = -float(polygon.find('./Attributes/Rotation').text)
            rr, cc = draw.polygon(r, c)
            img_polygon[rr, cc] = 1
            centerX = round(np.min(cc) + 0.5*(np.max(cc)-np.min(cc)))
            centerY = round(np.min(rr) + 0.5*(np.max(rr)-np.min(rr)))
            img_polygon = transform.rotate(img_polygon, angle=rotation, center=(centerX,centerY), order=0)
            rr, cc = np.where(img_polygon==1)
        except:   
            rr, cc  = draw.polygon(r, c)
            img_polygon[rr, cc ] = 1

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_polygon)
            case "#FF00FF00":
                masks_gleason3.append(img_polygon)
            case "#FFFF6600":
                masks_gleason4.append(img_polygon)
            case "#FFCC4E00":
                masks_cribriform.append(img_polygon)  
            case "#FFFF00FF":
                masks_glomeruloid.append(img_polygon)
            case "#FF990000":
                masks_gleason5.append(img_polygon)
    
    #Contour_spline
    for bezier in root.iter('Bezier'):
        try:
            color = bezier.find('./Attributes/Stroke').text
        except:
            continue
        points = bezier.find('./Geometry/Points').text
        array = np.array(np.mat(points.replace(' ', ';'))*scale, dtype=np.int64)
        r = array[:,1]
        c = array[:,0]
        img_bezier = np.zeros((img_height, img_width), dtype=np.int64)
        try:
            rotation = -float(bezier.find('./Attributes/Rotation').text)
            rr, cc = draw.polygon(r, c)
            img_bezier[rr, cc] = 1
            centerX = round(np.min(cc) + 0.5*(np.max(cc)-np.min(cc)))
            centerY = round(np.min(rr) + 0.5*(np.max(rr)-np.min(rr)))
            img_bezier = transform.rotate(img_bezier, angle=rotation, center=(centerX,centerY), order=0)
            rr, cc = np.where(img_bezier==1)
        except:   
            rr, cc  = draw.polygon(r, c)
            img_bezier[rr, cc ] = 1

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_bezier)
            case "#FF00FF00":
                masks_gleason3.append(img_bezier)
            case "#FFFF6600":
                masks_gleason4.append(img_bezier)
            case "#FFCC4E00":
                masks_cribriform.append(img_bezier)
            case "#FFFF00FF":
                masks_glomeruloid.append(img_bezier)
            case "#FF990000":
                masks_gleason5.append(img_bezier)

    #Contour with holes
    for bezier in root.iter('MultiPolygon'):
        try:
            color = bezier.find('./Attributes/Stroke').text
        except:
            continue
        points_big = bezier.find('./Geometry/Points').text
        points_small = bezier.findall('ChildPolygon')
        array_big = np.array(np.mat(points_big.replace(' ', ';'))*scale, dtype=np.int64)
        r_big = array_big[:,1]
        c_big = array_big[:,0]
        img_big = np.zeros((img_height, img_width), dtype=np.int64)
        rr_big, cc_big  = draw.polygon(r_big, c_big)
        img_big[rr_big, cc_big ] = 1
        img_small = np.zeros((img_height, img_width), dtype=np.int64)
        for i in range(len(points_small)):
            points = points_small[i].find('Geometry/Points').text
            array_small = np.array(np.mat(points.replace(' ', ';'))*scale, dtype=np.int64)
            r_small = array_small[:,1]
            c_small = array_small[:,0]
            rr_small, cc_small  = draw.polygon(r_small, c_small)
            img_small[rr_small, cc_small ] = 1
        img_big[img_small == 1] = 0 

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_big)
            case "#FF00FF00":
                masks_gleason3.append(img_big)
            case "#FFFF6600":
                masks_gleason4.append(img_big)
            case "#FFCC4E00":
                masks_cribriform.append(img_big) 
            case "#FFFF00FF":
                masks_glomeruloid.append(img_big)
            case "#FF990000":
                masks_gleason5.append(img_big)

    # circle
    for circle in root.iter('Circle'):
        try:
            color = circle.find('./Attributes/Stroke').text
        except:
            continue
        centerX = round(float(circle.find('./Geometry/CenterX').text)*scale)
        centerY = round(float(circle.find('./Geometry/CenterY').text)*scale)
        radius = round(float(circle.find('./Geometry/Radius').text)*scale)
        img_circle = np.zeros((img_height, img_width), dtype=np.int64)
        rr, cc = draw.disk((centerY, centerX ), radius)
        img_circle[rr, cc ] = 1

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_circle)
            case "#FF00FF00":
                masks_gleason3.append(img_circle)
            case "#FFFF6600":
                masks_gleason4.append(img_circle)
            case "#FFCC4E00":
                masks_cribriform.append(img_circle)
            case "#FFFF00FF":
                masks_glomeruloid.append(img_circle)
            case "#FF990000":
                masks_gleason5.append(img_circle)
    
    # ellipse
    for ellipse in root.iter('Ellipse'): 
        try:
            color = ellipse.find('./Attributes/Stroke').text
        except:
            continue
        centerX = round(float(ellipse.find('./Geometry/CenterX').text)*scale)
        centerY = round(float(ellipse.find('./Geometry/CenterY').text)*scale)
        radiusX = round(float(ellipse.find('./Geometry/RadiusX').text)*scale)
        radiusY = round(float(ellipse.find('./Geometry/RadiusY').text)*scale)
        img_ellipse = np.zeros((img_height, img_width),dtype=np.int64)
        try:
            rotation = -float(ellipse.find('./Attributes/Rotation').text)
            rr, cc = draw.ellipse(centerY, centerX , radiusY, radiusX)
            img_ellipse[rr, cc] = 1
            img_ellipse = transform.rotate(img_ellipse, angle=rotation, center=(centerX,centerY), order=0)
            rr, cc = np.where(img_ellipse==1)
        except:
            rr, cc = draw.ellipse(centerY, centerX , radiusY, radiusX)
            img_ellipse[rr, cc] = 1

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_ellipse)
            case "#FF00FF00":
                masks_gleason3.append(img_ellipse)
            case "#FFFF6600":
                masks_gleason4.append(img_ellipse)
            case "#FFCC4E00":
                masks_cribriform.append(img_ellipse) 
            case "#FFFF00FF":
                masks_glomeruloid.append(img_ellipse)
            case "#FF990000":
                masks_gleason5.append(img_ellipse)

    # rectangle
    for rectangle in root.iter('Rectangle'):
        try:
            color = rectangle.find('./Attributes/Stroke').text
        except:
            continue
        left = round(float(rectangle.find('./Geometry/Left').text)*scale)
        top = round(float(rectangle.find('./Geometry/Top').text)*scale)
        width = round(float(rectangle.find('./Geometry/Width').text)*scale)
        height = round(float(rectangle.find('./Geometry/Height').text)*scale)
        img_rectangle = np.zeros((img_height, img_width),dtype=np.int64)
        try:
            rotation = -float(rectangle.find('./Attributes/Rotation').text)
            rr, cc = draw.rectangle((top,left), end=(top+height, left+width))
            img_rectangle[rr, cc] = 1
            centerX = round(left+0.5*width)
            centerY = round(top+0.5*height)
            img_rectangle = transform.rotate(img_rectangle, angle=rotation, center=(centerX,centerY), order=0)
            rr, cc = np.where(img_rectangle==1)
        except:
            rr, cc = draw.rectangle((top,left), end=(top+height, left+width)) 
            img_rectangle[rr, cc] = 1

        #annotations[label] = (rr, cc )
        match color:
            case "#FF0000FF":
                masks_healthy.append(img_rectangle)
            case "#FF00FF00":
                masks_gleason3.append(img_rectangle)
            case "#FFFF6600":
                masks_gleason4.append(img_rectangle)
            case "#FFCC4E00":
                masks_cribriform.append(img_rectangle) 
            case "#FFFF00FF":
                masks_glomeruloid.append(img_rectangle)
            case "#FF990000":
                masks_gleason5.append(img_rectangle)

    masks = [masks_healthy, masks_gleason3, masks_gleason4, masks_cribriform, masks_glomeruloid, masks_gleason5]
    return masks

In [18]:
full_masks = []

for i in range(len(lst)):
    img = AICSImage("/scratch/c.c23115040/H&E_Annotated/" + lst[i])
    #breakpoint()
    scale = 1
    img_width =round(img.shape[4]*scale)
    img_height = round(img.shape[3]*scale)
    img_data = np.zeros((img_height, img_width, 3))
    for c in range(3):
        img_data[:,:, c] = transform.rescale(img.get_image_data("YX", S=c), scale, anti_aliasing=False)
    masks = extract_annotations_from_metadata(img.metadata, scale)  
    annotations = []
    for j in range(6):
        match len(masks[j]):
            case 0:
                annotations.append(np.zeros((img_height, img_width)).astype(bool))
            case 1:
                annotations.append(np.array(masks[j]).astype(bool)[0,:,:])
            case _ if len(masks[j]) >= 2:
                annotations.append(np.logical_or.reduce(np.array(masks[j])))
    final_mask = np.zeros((img_height, img_width, 1))
    final_mask[annotations[0] == 1] = 1
    final_mask[annotations[1] == 1] = 2
    final_mask[annotations[2] == 1] = 3
    final_mask[annotations[3] == 1] = 4
    final_mask[annotations[4] == 1] = 5 
    final_mask[annotations[5] == 1] = 6
    full_masks.append(final_mask)             

In [19]:
for i in range(len(lst)):
    full_masks[i] = full_masks[i].astype(np.uint8)

In [20]:
np.save("/home/c.c23115040/full_masks.npy", np.array(full_masks, dtype=object), allow_pickle=True)

In [ ]:
full_masks = []

for i in range(len(lst2)):
    img = AICSImage("/scratch/c.c23115040/H&E_Annotated2/" + lst2[i] + ".czi")
    #breakpoint()
    scale = 1
    img_width =round(img.shape[4]*scale)
    img_height = round(img.shape[3]*scale)
    img_data = np.zeros((img_height, img_width, 3))
    for c in range(3):
        img_data[:,:, c] = transform.rescale(img.get_image_data("YX", S=c), scale, anti_aliasing=False)
    masks = extract_annotations_from_metadata(img.metadata, scale)  
    annotations = []
    for j in range(6):
        match len(masks[j]):
            case 0:
                annotations.append(np.zeros((img_height, img_width)).astype(bool))
            case 1:
                annotations.append(np.array(masks[j]).astype(bool)[0,:,:])
            case _ if len(masks[j]) >= 2:
                annotations.append(np.logical_or.reduce(np.array(masks[j])))
    final_mask = np.zeros((img_height, img_width, 1))
    final_mask[annotations[0] == 1] = 1
    final_mask[annotations[1] == 1] = 2
    final_mask[annotations[2] == 1] = 3
    final_mask[annotations[3] == 1] = 4
    final_mask[annotations[4] == 1] = 5 
    final_mask[annotations[5] == 1] = 6
    full_masks.append(final_mask)             

In [60]:
for i in range(len(lst2)):
    full_masks[i] = full_masks[i].astype(np.uint8)

In [61]:
np.save("/home/c.c23115040/full_masks2.npy", np.array(full_masks, dtype=object), allow_pickle=True)

In [7]:
full_masks = []

for i in range(len(lst3)):
    img = AICSImage("/scratch/c.c23115040/WCB_Sequential_Annotated/" + lst3[i])
    #breakpoint()
    scale = 1
    img_width =round(img.shape[4]*scale)
    img_height = round(img.shape[3]*scale)
    img_data = np.zeros((img_height, img_width, 3))
    for c in range(3):
        img_data[:,:, c] = transform.rescale(img.get_image_data("YX", S=c), scale, anti_aliasing=False)
    masks = extract_annotations_from_metadata(img.metadata, scale)  
    annotations = []
    for j in range(6):
        match len(masks[j]):
            case 0:
                annotations.append(np.zeros((img_height, img_width)).astype(bool))
            case 1:
                annotations.append(np.array(masks[j]).astype(bool)[0,:,:])
            case _ if len(masks[j]) >= 2:
                annotations.append(np.logical_or.reduce(np.array(masks[j])))
    final_mask = np.zeros((img_height, img_width, 1))
    final_mask[annotations[0] == 1] = 1
    final_mask[annotations[1] == 1] = 2
    final_mask[annotations[2] == 1] = 3
    final_mask[annotations[3] == 1] = 4
    final_mask[annotations[4] == 1] = 5 
    final_mask[annotations[5] == 1] = 6
    full_masks.append(final_mask)             

In [8]:
for i in range(len(lst3)):
    full_masks[i] = full_masks[i].astype(np.uint8)

In [9]:
np.save("/home/c.c23115040/full_masks3.npy", np.array(full_masks, dtype=object), allow_pickle=True)

In [ ]:
full_masks = []

for i in range(len(lst4)):
    img = AICSImage("/scratch/c.c23115040/MGP/D&E Annotated/" + lst4[i] + ".czi")
    #breakpoint()
    scale = 1
    img_width =round(img.shape[4]*scale)
    img_height = round(img.shape[3]*scale)
    img_data = np.zeros((img_height, img_width, 3))
    for c in range(3):
        img_data[:,:, c] = transform.rescale(img.get_image_data("YX", S=c), scale, anti_aliasing=False)
    masks = extract_annotations_from_metadata(img.metadata, scale)  
    annotations = []
    for j in range(6):
        match len(masks[j]):
            case 0:
                annotations.append(np.zeros((img_height, img_width)).astype(bool))
            case 1:
                annotations.append(np.array(masks[j]).astype(bool)[0,:,:])
            case _ if len(masks[j]) >= 2:
                annotations.append(np.logical_or.reduce(np.array(masks[j])))
    final_mask = np.zeros((img_height, img_width, 1))
    final_mask[annotations[0] == 1] = 1
    final_mask[annotations[1] == 1] = 2
    final_mask[annotations[2] == 1] = 3
    final_mask[annotations[3] == 1] = 4
    final_mask[annotations[4] == 1] = 5 
    final_mask[annotations[5] == 1] = 6
    full_masks.append(final_mask) 
    print(i)

In [7]:
for i in range(len(lst4)):
    full_masks[i] = full_masks[i].astype(np.uint8)

In [8]:
np.save("/home/c.c23115040/full_masks4.npy", np.array(full_masks, dtype=object), allow_pickle=True)